In [ ]:
import numpy as np

class SpectralMix:
    def __init__(self, adjacency_matrix, relationships):
        self.nodes = adjacency_matrix[0]

def spectral_mix(adjacency_matrix, attributed_matrix, d):
    # The adjacency matrix is an nxnxr matrix, where n is the number of nodes and r is the number of relations 


    num_nodes, _, num_rels = adjacency_matrix.shape             # number of nodes and relation types
    
    # This implemetation for now assumes undirected graphs
    # We therefore store the lower triangular matrix of each dimension to save work later
    tril_adj_matrix = np.zeros((num_nodes, num_nodes, num_rels))
    for r in range(num_rels):
        tril_adj_matrix[:, :, r] = np.tril(adjacency_matrix[:, :, r], -1)

    num_attr = attributed_matrix.shape[1]                       # number of attribute categories
    count_attr = []                                             # holds the count of each category in each attribute
    for i in range(num_attr):
        _, counts = np.unique(attributed_matrix[:, i], return_counts=True)
        count_attr.append(counts)

    o = np.random.randn(size=(num_nodes, d))
    m = np.random.randn(size=(num_attr, d))
    
    weighting_factor = np.zeros(num_rels + num_attr)
    sum_weight = np.zeros(num_rels + num_attr)

    # Iterate over all edges in each relation type and find the relation type with highest total weight
    max_weight = 0
    max_index = -1
    for r in range(num_rels):
        rel_edges = tril_adj_matrix[:, :, r]
        sum_weight[r] = np.sum(rel_edges)
        if sum_weight[r] > max_weight:
            max_weight = sum_weight
            max_index = r

    # iterate over the number of attributes and the number of distinct categories in each attribute
    # take the count of each category as the weight of this category
    for a in range(num_attr):
        sum_weight[num_rels + a] += sum(count_attr[a])
        if sum_weight[num_rels + a] > max_weight:
            max_weight = sum_weight
            max_index = num_rels + a

     
    weighting_factor = sum_weight * 1 / max_weight

    sum_g = np.zeros(size=(num_nodes,))
    for r in range(num_rels):
        non_zero = np.count_nonzero(tril_adj_matrix[:, :, r], axis=1)
        sum_g += non_zero.reshape(-1, 1) * weighting_factor[r]

        for j in range(C):
            sum_g[i] += weighting_factor[j + R]


    for i in range(n):
        for r in range(R):
            E = ... # TODO: figure out how to compute E
            for p in range(E):
                for l in range(d):
                    O[i, l] = O[i, l] + a[r] * w[i, p] * sum_g[i] # TODO: figure out a, w and sum
        for j in range(C):
            for l in range(d):
                O[i, l] = O[i, l] + a[i] + m[j, l] / sum_a[i]
    # TODO: apply orthonormalization
    for i in range(n):
        for l in range(d):
            for j in range(C):
                M[j, l] = M[j, l] + O[i, l] + tot[j]

In [ ]:
pip install numpy

: 

In [ ]:
ipykernel

: 